In [1]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet, 'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [6]:
test_dataset

,text,labels
0,This simple household item saves lives,True
1,You'll Never Believe Who Robert Downey Jr.'s '...,True
2,THE NEWEST THEORY ON HIDDLESWIFT'S ROMANCE IS ...,False
3,"The little girl from ""Titanic"" is all grown up...",True
4,Why we never really get over that first love,False
...,...,...
821,Inside our three-month effort to attend Apple'...,False
822,Dad And Son Are Seconds From Assassination By ...,False
823,Kanye West Reveals His Favorite Song... Of All...,False
824,"""Pippi Longstocking"" star arrested",True


In [7]:
train_dataset

,text,labels
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [8]:
validation_dataset

,text,labels
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
652,"Dog Dies One Hour After Hiking With His Owner,...",False
653,This is what happens when you leave a hotel cl...,False
654,This Texas GOP elector announces that he won't...,True
655,WikiLeaks' Julian Assange Reported Dead,False


# Training

In [3]:
configurations = []

for learn_rate in [1e-5, 4e-5, 1e-4]:
    for warumup_ratio in [0.02, 0.06, 0.1]:
        for batch_size in [8, 16, 32]:
            configurations += [{
                "overwrite_output_dir": True,
                "num_train_epochs": 10,
                "fp16": False,
                "train_batch_size": batch_size,
                "gradient_accumulation_steps": 4,
                "evaluate_during_training": True,
                "max_seq_length": 64,
                "learning_rate": learn_rate,
                "early_stopping_consider_epochs": True,
                "early_stopping_delta": 0.01,
                "early_stopping_metric": "acc",
                "early_stopping_metric_minimize": False,
                "early_stopping_patience": 3,
                "evaluate_during_training_steps": 331,
                "output_dir": "outputs/",
                'warmup_ratio': warumup_ratio,
                # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
                #                    # <> überschreiben sich gegenseitig
                'save_steps': 2000,
                "manual_seed": 12345
            }]

In [7]:
def train_model(config, num):
    from simpletransformers.classification import ClassificationModel
    import sklearn
    
    config["output_dir"] = "outputs/deberta_" + str(num)
    
    model = ClassificationModel("deberta", "microsoft/deberta-large", args = config)
    # model.train_model(train_dataset)
    model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [ ]:
for config, num in zip(configurations, range(len(configurations))):
    print('Running configuration number', num)
    
    train_model(config, num)

Running configuration number 0


Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.bias

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

...

Running Epoch 1 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

In [4]:
from simpletransformers.classification import ClassificationModel
import sklearn
model = ClassificationModel("deberta", "microsoft/deberta-large", args = {
                "output_dir": "outputs/test",
                "overwrite_output_dir": True,
                "num_train_epochs": 5,
                "fp16": False,
                "train_batch_size": 8,
                "gradient_accumulation_steps": 4,
                "evaluate_during_training": True,
                "max_seq_length": 64,
                "learning_rate": 4e-5,
                "early_stopping_consider_epochs": True,
                "early_stopping_delta": 0.01,
                "early_stopping_metric": "acc",
                "early_stopping_metric_minimize": False,
                "early_stopping_patience": 3,
                "evaluate_during_training_steps": 331,
                "output_dir": "outputs/",
                'warmup_ratio': 0.06,
                # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
                #                    # <> überschreiben sich gegenseitig
                'save_steps': 331,
                "manual_seed": 12345
            })
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Downloading:   0%|          | 0.00/475 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.bias

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

(410,
 defaultdict(list,
             {'global_step': [82, 164, 246, 328, 331, 410],
              'train_loss': [0.07401979714632034,
               0.1318412572145462,
               0.010398376733064651,
               0.005076257977634668,
               0.015512799844145775,
               0.33082717657089233],
              'mcc': [0.5271660583389545,
               0.5810383897390848,
               0.5250945882122984,
               0.5779053664598808,
               0.5462046824907045,
               0.5452048210015293],
              'tp': [236, 231, 265, 257, 265, 277],
              'tn': [264, 284, 236, 261, 243, 230],
              'fp': [58, 38, 86, 61, 79, 92],
              'fn': [99, 104, 70, 78, 70, 58],
              'auroc': [0.8347733382775564,
               0.8461388708630759,
               0.8368313710948365,
               0.8465931213497728,
               0.8429313062019097,
               0.8374524891072588],
              'auprc': [0.8562279644365282,
   

In [38]:
model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

({'mcc': 0.5189907816932701,
  'tp': 264,
  'tn': 235,
  'fp': 87,
  'fn': 71,
  'auroc': 0.8332344488736442,
  'auprc': 0.8547997162455769,
  'acc': 0.7595129375951294,
  'eval_loss': 1.5449147986592473},
 array([[-4.50758696,  4.8231678 ],
        [-4.62452698,  4.63890123],
        [ 4.56198597, -4.33566332],
        ...,
        [ 0.98726308, -0.87757879],
        [ 3.27670145, -3.22399449],
        [-4.55029869,  4.87138462]]),
 [['Five Nights at Freddy’s Sequel Delayed for Weird Reason',
   'Here’s how much you should be tipping your hairdresser',
   "A man swallowed a microSD card and you won't believe what happened next!",
   'This popular soda could cure your hangovers scientists say:',
   "The anytime snack you won't feel guilty about eating",
   'You won\'t believe this stunning "Harry Potter" revelation about Professor McGonagall',
   "J.J. Abrams has an answer on if there will be a post-credits scene in the new 'Star Wars'",
   'Kristin Cavallari (@KristinCav) opens up abo

In [37]:
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

({'mcc': 0.5662373938122638,
  'tp': 338,
  'tn': 309,
  'fp': 94,
  'fn': 85,
  'auroc': 0.83468548533751,
  'auprc': 0.8406305749647641,
  'acc': 0.7832929782082324,
  'eval_loss': 1.4019140609235854},
 array([[-4.73007059,  4.75768614],
        [-4.66822481,  4.89313078],
        [ 4.12303448, -3.80747771],
        ...,
        [-4.3079958 ,  4.59369469],
        [-4.60104084,  4.89709044],
        [-4.45188284,  4.57753611]]),
 [])

# Model Selection with Validation data

In [3]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm
import os.path


for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/deberta_" + str(num) + "/"
    
    df = []
    
    if os.path.isfile(directory + 'eval-results.jsonl'):
        print('Skip ', num)
        continue
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        # print(checkpoint)
        model = ClassificationModel("deberta", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
    
    pd.DataFrame(df).to_json(directory + 'eval-results.jsonl')



Skip  0


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

 30%|███       | 3/10 [02:23<05:35, 47.92s/it]


RuntimeError: CUDA out of memory. Tried to allocate 198.00 MiB (GPU 0; 39.59 GiB total capacity; 16.65 GiB already allocated; 124.69 MiB free; 16.66 GiB reserved in total by PyTorch)

In [5]:
import pandas as pd

df = []
for config, num in zip(configurations, range(len(configurations))):
    df += [pd.read_json("outputs/deberta_" + str(num) + "/eval-results.jsonl")]
    
df = pd.concat(df)
df

,checkpoint,valid_acc,test_acc,config
0,outputs/deberta_0/checkpoint-246-epoch-3,0.779300,0.778450,"{'overwrite_output_dir': True, 'num_train_epoc..."
1,outputs/deberta_0/checkpoint-656-epoch-8,0.768645,0.750605,"{'overwrite_output_dir': True, 'num_train_epoc..."
2,outputs/deberta_0/checkpoint-820-epoch-10,0.786910,0.755448,"{'overwrite_output_dir': True, 'num_train_epoc..."
3,outputs/deberta_0/checkpoint-410-epoch-5,0.779300,0.766344,"{'overwrite_output_dir': True, 'num_train_epoc..."
4,outputs/deberta_0/checkpoint-331,0.756469,0.755448,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
5,outputs/deberta_26/checkpoint-80-epoch-4,0.770167,0.773608,"{'overwrite_output_dir': True, 'num_train_epoc..."
6,outputs/deberta_26/checkpoint-200-epoch-10,0.788432,0.771186,"{'overwrite_output_dir': True, 'num_train_epoc..."
7,outputs/deberta_26/checkpoint-180-epoch-9,0.786910,0.755448,"{'overwrite_output_dir': True, 'num_train_epoc..."
8,outputs/deberta_26/checkpoint-20-epoch-1,0.657534,0.690073,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [ ]:
df.to_json('outputs/deberta_eval_results.csv')

In [5]:
len(configurations)

27

In [ ]:
print('finished!')

In [ ]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

In [6]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.8066971081000001 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmup_ratio': 0.02, 'save_steps': 2000, 'manual_seed': 12345}
0.797564688 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 16, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmup_ratio'